In [1]:
import geopandas as gpd
import os
from glob import glob
import pandas as pd
import requests
import zipfile
import time
import json
import shapely

In [2]:
project_folder_path = r"/work/mhealthresearchgroup/Geofabrik"

In [3]:
download_folder_path = os.path.join(project_folder_path, "download")

In [4]:
unzipped_folder_path = os.path.join(project_folder_path, "unzipped")

In [5]:
organized_data_folder_path = os.path.join(project_folder_path, "organized_landmarks")

In [6]:
combined_data_folder_path = os.path.join(project_folder_path, "organized_landmarks_combined")

In [7]:
geo_map_path = "/home/li.jix/repo/geofabrik_landmark_map/map/Amenity List - combined_GF_cleaned.csv"

In [8]:
base_url = "http://download.geofabrik.de/north-america/us/"

In [9]:
update = False

In [10]:
# create root folder
def create_folder(p):
    if not os.path.exists(p):
        os.makedirs(p)

### 1. Download, unzip, and store Geofabrik data from all states

In [11]:
# download
def get_shp_file_link(state):
    if state == 'united states virgin islands':
        state_name = "us-virgin-islands"
    elif state == 'norcal' or state == 'socal':
        state_name = "california/"+state
    else:
        state_name = state.replace(" ", "-")
    return base_url + state_name + "-latest-free.shp.zip"


In [12]:
def download_state_zipfile(state, url):
    r = requests.get(url, allow_redirects=True)
    save_path = os.path.join(download_folder_path, state+'.shp.zip')
    open(save_path, 'wb').write(r.content)
    return save_path

In [13]:
def unzip_file(zipfile_path, zip_to_path):
    with zipfile.ZipFile(zipfile_path, 'r') as zip_ref:
        zip_ref.extractall(zip_to_path)

In [14]:
l = "alabama, alaska, arizona, arkansas, norcal, socal, colorado, connecticut, delaware, district of columbia, florida, georgia, hawaii, idaho, illinois, indiana, iowa, kansas, kentucky, louisiana, maine, maryland, massachusetts, michigan, minnesota, mississippi, missouri, montana, nebraska, nevada, new hampshire, new jersey, new mexico, new york, north carolina, north dakota, ohio, oklahoma, oregon, pennsylvania, puerto rico, rhode island, south carolina, south dakota, tennessee, texas, united states virgin islands, utah, vermont, virginia, washington, west virginia, wisconsin, wyoming"
state_list = l.split(", ")

In [15]:
start = time.time()
print("=================")
print("Download Geofabrik data for all states to {}".format(download_folder_path))
print("Unzip Geofabrik data for all states to {}".format(unzipped_folder_path))
print("=================")

create_folder(download_folder_path)
create_folder(unzipped_folder_path)
for state in state_list:
    print(state)
    # update or not
    zip_to_path = os.path.join(unzipped_folder_path, state)
    if not update: # skip state if unzipped file exists
        if os.path.exists(zip_to_path):
            print("skip because unzipped file exists")
            continue
        
    
    # download
    url = get_shp_file_link(state)
    download_save_path = download_state_zipfile(state, url)
    print("    "+url)
    print("    "+"Downloaded to : " + download_save_path)
    
    # unzip
    zipfile_path = os.path.join(download_folder_path, state + '.shp.zip')
    unzip_file(zipfile_path, zip_to_path)
    print("    "+"Unzipped to : " + zip_to_path)
    
end = time.time()
print(f"Runtime of the program is {end - start}")

Download Geofabrik data for all states to /work/mhealthresearchgroup/Geofabrik/download
Unzip Geofabrik data for all states to /work/mhealthresearchgroup/Geofabrik/unzipped
alabama
skip because unzipped file exists
alaska
skip because unzipped file exists
arizona
skip because unzipped file exists
arkansas
skip because unzipped file exists
norcal
skip because unzipped file exists
socal
skip because unzipped file exists
colorado
skip because unzipped file exists
connecticut
skip because unzipped file exists
delaware
skip because unzipped file exists
district of columbia
skip because unzipped file exists
florida
skip because unzipped file exists
georgia
skip because unzipped file exists
hawaii
skip because unzipped file exists
idaho
skip because unzipped file exists
illinois
skip because unzipped file exists
indiana
skip because unzipped file exists
iowa
    http://download.geofabrik.de/north-america/us/iowa-latest-free.shp.zip
    Downloaded to : /work/mhealthresearchgroup/Geofabrik/down

In [44]:
923/60

15.383333333333333

### 2. Parse Geofabrik map

In [15]:
# first run "clean_csv_map" and use "Amenity List - combined_GF_clean.csv"

In [15]:
# convert csv map to json map
map_dict = {"unique_list": [], "map": {}}

with open(geo_map_path) as fp:
    Lines = fp.readlines()
    for line in Lines:  # skip head line
        line_list = line.split(",")
        label_1st = line_list[0]
        label_2nd = line_list[1]
        label_3rd = line_list[2]
        if len(label_1st) > 0:
            level1 = label_1st
            map_dict['map'][level1] = dict()
            map_dict['unique_list'].append(label_1st) 
        if len(label_2nd) > 0:
            level2 = label_2nd
            map_dict['map'][level1][level2] = dict()
            map_dict['unique_list'].append(label_2nd) 
        if len(label_3rd) > 0:
            line_list_lvl3 = [x for x in line_list if (len(x) > 0) and (x != '\n')]
            map_dict['map'][level1][level2][label_3rd] = line_list_lvl3[1:]
            map_dict['unique_list']+= line_list_lvl3
        

In [16]:
map_dict

{'unique_list': ['residential',
  'residential',
  'apartment',
  'studio',
  'dormitory',
  'dorm',
  'fraternity_house',
  'fraternity',
  'house',
  'carriage_house',
  'deckhouse',
  'semidetached_house',
  'townhouse',
  'condominium',
  'condo',
  'condominum',
  'affordable_housing',
  'pavillion',
  'residential',
  'bungalow',
  'cabin',
  'hut',
  'lodge',
  'mansion',
  'mobile_home',
  'residence',
  'static_caravan',
  'group_housing',
  'housing',
  'home',
  'building',
  'building',
  'historic_building',
  'county_building',
  'department_building',
  'music_building',
  'skyscraper',
  'cultural_center',
  'engineering_building',
  'organization',
  'maintenance_building',
  'service',
  'education',
  'college',
  'college_building',
  'childcare',
  'childcare_facility',
  'day_care',
  'preschool',
  'kindergarten',
  'school',
  'high_school',
  'music_school',
  'classroom',
  'library',
  'university_library',
  'university',
  'education',
  'academic',
  'acad

### 3. Extract and store Geofabrik data based on label map

In [17]:
layers = ["buildings", "landuse", "natural", "places", "pofw", "pois", "railways", "roads", "traffic", "transport", "water", "waterways"]

In [19]:

# def extract_data_for_label(labels, lvl3_path, category):
#     if category == "point":
#         suffix = "_free_*.shp"
#         filename_suffix = "_point"
#     elif category == "polygon":
#         suffix = "_a_free_*.shp"
#         filename_suffix = "_polygon"
#     else:
#         print("wrong type of file requested")
#         return
    
#     layer_df = pd.DataFrame()
#     for layer in layers:
#         for state in state_list:
#             state_folder_path = os.path.join(unzipped_folder_path, state)
#             state_shapefile_path = os.path.join(state_folder_path, "gis_osm_" + layer + suffix)
#             shapefile_finding_list = list(glob(state_shapefile_path))
#             if len(shapefile_finding_list) > 1:
#                 print("{} shapefiles found for {} for state {}".format(len(shapefile_finding_list), labels, state))

#             for p_shapefile in shapefile_finding_list:
#                 shpfile_df = gpd.read_file(p_shapefile)
#                 column_to_check = ["fclass", "type"]
#                 column_to_check = list(set(shpfile_df.columns).intersection(column_to_check))

                
#                 for label in labels:
#                     for colname in column_to_check:
#                         shpfile_df_label = shpfile_df[shpfile_df[colname] == label]
#                         layer_df = layer_df.append(shpfile_df_label)

#     # save extraction to shapefile
#     if layer_df.shape[0] > 0:
#         layer_df.to_file(lvl3_path + os.sep + labels[0] + filename_suffix + ".shp")
#     else:
#         print("                  No {} file found for {}".format(category, labels[0]))

# # extract function
# def organize_data_for_label(lvl3_label, label_map_lvl2, lvl3_path):
#     synonyms = label_map_lvl2[lvl3_label]
#     labels = [x for x in synonyms]
#     labels.insert(0, lvl3_label)
#     # point
#     extract_data_for_label(labels, lvl3_path, "point")
#     # polygon
#     extract_data_for_label(labels, lvl3_path, "polygon")

# # extract pipeline
# label_map = map_dict['map']
# create_folder(organized_data_folder_path)
# for lvl1_label in label_map: # from top level to bottom level
#     print("Level 1 : "+lvl1_label)
#     lvl1_path = organized_data_folder_path + os.sep + lvl1_label
#     create_folder(lvl1_path)
#     for lvl2_label in label_map[lvl1_label]:
#         print("  Level 2 : "+lvl2_label)
#         lvl2_path = lvl1_path + os.sep + lvl2_label
#         create_folder(lvl2_path)
#         for lvl3_label in label_map[lvl1_label][lvl2_label]:
#             print("    Level 3 : "+lvl3_label)
#             lvl3_path = lvl2_path + os.sep + lvl3_label
#             if not update:
#                 if len(os.listdir(path)) > 0:
#                     continue
#             create_folder(lvl3_path)
#             organize_data_for_label(lvl3_label, label_map[lvl1_label][lvl2_label], lvl3_path)

In [20]:
# extract 
# state - point/polygon - layer - label

In [18]:
def clean_tag_column(tag_series):
    
    cleaned_series = tag_series.str.lower()
    cleaned_series = cleaned_series.str.replace(" ", "_", case = False)
    cleaned_series = pd.Series([x[:-1] if x.endswith('s') else x for x in cleaned_series])
    
    return cleaned_series

In [19]:
def extract_all_landmarks_from_shapefile(label_map, organized_data_folder_path, shpfile_df_single_tag, shpfile_df_multi_tag, column_to_check, state, category, layer, shpfile_num):
    included_num = 0
    for lvl1_label in label_map: # from top level to bottom level
    #     print("Level 1 : "+lvl1_label)
        lvl1_path = organized_data_folder_path + os.sep + lvl1_label
        create_folder(lvl1_path)
        for lvl2_label in label_map[lvl1_label]:
    #         print("  Level 2 : "+lvl2_label)
            lvl2_path = lvl1_path + os.sep + lvl2_label
            label_map_lvl2 = label_map[lvl1_label][lvl2_label]
            create_folder(lvl2_path)
            for lvl3_label in label_map[lvl1_label][lvl2_label]:
    #             print("    Level 3 : "+lvl3_label)
                lvl3_path = lvl2_path + os.sep + lvl3_label
                lvl3_file_name = "_".join([lvl3_label, state, layer, shpfile_num, category+".shp"])
                lvl3_file_path = os.path.join(lvl3_path, lvl3_file_name)
                if not update:
                    if os.path.exists(lvl3_file_path):
                        continue
                create_folder(lvl3_path)
                synonyms = label_map_lvl2[lvl3_label]
                labels = [x for x in synonyms]
                labels.insert(0, lvl3_label)

                df_label = pd.DataFrame()
                for label in labels:
                    shpfile_df_single_tag_label = shpfile_df_single_tag[shpfile_df_single_tag[column_to_check] == label]
                    shpfile_df_multi_tag_label = shpfile_df_multi_tag[shpfile_df_multi_tag[column_to_check].str.contains(label)]
                    df_label = df_label.append(shpfile_df_single_tag_label)
                    df_label = df_label.append(shpfile_df_multi_tag_label)

                # save extraction to shapefile
                if df_label.shape[0] > 0:
                    df_label.to_file(lvl3_file_path)
                    included_num += df_label.shape[0]
    return included_num

In [20]:
start = time.time()

#
label_map = map_dict['map']
create_folder(organized_data_folder_path)

#
label_count_dict = dict()
for layer in layers:
    label_count_dict[layer] = {"none": 0, "labeled": 0, "included": 0}

#
start = 0
for state in state_list:
    print(state)
    if state == "new jersey":
        start = 1
    if start == 0:
        continue
    start_state = time.time()
    state_folder_path = os.path.join(unzipped_folder_path, state)
    for category in ["point", "polygon"]:
        print("  "+category)
        if category == "point":
            suffix = "_free_*.shp"
            filename_suffix = "_point"
        elif category == "polygon":
            suffix = "_a_free_*.shp"
            filename_suffix = "_polygon"
        else:
            pass
        
        for layer in layers:          
            print("    "+layer)
            shapefile_pattern = os.path.join(state_folder_path, "gis_osm_" + layer + suffix)
            shapefile_finding_list = list(glob(shapefile_pattern))
            if len(shapefile_finding_list) > 1:
                print("{} shapefiles found for {} for state {}".format(len(shapefile_finding_list), labels, state))

            for p_shapefile in shapefile_finding_list:
                shpfile_df = gpd.read_file(p_shapefile)
                label_count_dict[layer]["none"] += shpfile_df.shape[0]
                column_to_check = ["fclass", "type"]
                column_to_check = list(set(shpfile_df.columns).intersection(column_to_check))
                if len(column_to_check) > 1:
                    if len(shpfile_df['fclass'].unique()) > len(shpfile_df['type'].unique()):
                        column_to_check = "fclass"
                    else:
                        column_to_check = "type"
                else:
                    column_to_check = column_to_check[0]
                print("      column to check : "+column_to_check)
#                 column_to_check = list(set(shpfile_df.columns).intersection(column_to_check))
                shpfile_num = p_shapefile.split("_free_")[1].strip(".shp")
    
                # clean label columns: lower case, replace space with underscore, remove ending s, split by ";"
                shpfile_df["tag_col_type"] = ["none" if x is None else "str" for x in shpfile_df[column_to_check]]
                shpfile_df = shpfile_df[shpfile_df["tag_col_type"]!="none"]
                label_count_dict[layer]["labeled"] += shpfile_df.shape[0]
                shpfile_df.reset_index(inplace=True, drop=True)
                shpfile_df[column_to_check] = clean_tag_column(shpfile_df[column_to_check])
                shpfile_df[column_to_check] = shpfile_df[column_to_check].astype(str)
                shpfile_df_single_tag = shpfile_df[~shpfile_df[column_to_check].str.contains(";")]
                shpfile_df_single_tag.reset_index(inplace=True, drop=True)
                shpfile_df_multi_tag = shpfile_df[shpfile_df[column_to_check].str.contains(";")]
                shpfile_df_multi_tag.reset_index(inplace=True, drop=True)
                
                included_num = extract_all_landmarks_from_shapefile(label_map, organized_data_folder_path, shpfile_df_single_tag, shpfile_df_multi_tag, column_to_check, state, category, layer, shpfile_num)
                label_count_dict[layer]["included"] += included_num
    end = time.time()
    print(f"Runtime for {state} is {end - start_state}")
                
end = time.time()
print(f"Total runtime of the program is {end - start}")

alabama
alaska
arizona
arkansas
norcal
socal
colorado
connecticut
delaware
district of columbia
florida
georgia
hawaii
idaho
illinois
indiana
iowa
kansas
kentucky
louisiana
maine
maryland
massachusetts
michigan
minnesota
mississippi
missouri
montana
nebraska
nevada
new hampshire
new jersey
  point
    buildings
    landuse
    natural
      column to check : fclass
    places
      column to check : fclass
    pofw
      column to check : fclass
    pois
      column to check : fclass
    railways
      column to check : fclass
    roads
      column to check : fclass
    traffic
      column to check : fclass
    transport
      column to check : fclass
    water
    waterways
      column to check : fclass
  polygon
    buildings
      column to check : type
    landuse
      column to check : fclass
    natural
      column to check : fclass
    places
      column to check : fclass
    pofw
      column to check : fclass
    pois
      column to check : fclass
    railways
    road

/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


    roads
      column to check : fclass
    traffic
      column to check : fclass
    transport
      column to check : fclass
    water
    waterways
      column to check : fclass
  polygon
    buildings
      column to check : type
    landuse
      column to check : fclass
    natural
      column to check : fclass
    places
      column to check : fclass
    pofw
      column to check : fclass
    pois
      column to check : fclass
    railways
    roads
    traffic
      column to check : fclass
    transport
      column to check : fclass
    water
      column to check : fclass
    waterways
Runtime for united states virgin islands is 100.48437261581421
utah
  point
    buildings
    landuse
    natural
      column to check : fclass
    places
      column to check : fclass
    pofw
      column to check : fclass
    pois
      column to check : fclass
    railways
      column to check : fclass
    roads
      column to check : fclass
    traffic
      column to check : f

/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


    transport
      column to check : fclass


...

    transport
      column to check : fclass


/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


    water
      column to check : fclass


/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
/home/li.jix/pyenvs/TIME/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


    waterways
Runtime for wyoming is 244.98759746551514
Total runtime of the program is 1655483844.0750072


In [84]:
map_file_path = os.path.abspath(os.path.join(geo_map_path, os.pardir))

label_count_path = os.path.join(map_file_path, "label_count_dict.json")

with open(label_count_path, 'w') as fp:
    json.dump(label_count_dict, fp)

In [25]:
10311.742320775986/60/60

2.8643728668822184

In [85]:
label_count_dict

{'buildings': {'none': 48435833, 'labeled': 16283105, 'included': 14588810},
 'landuse': {'none': 3344978, 'labeled': 3344978, 'included': 3331698},
 'natural': {'none': 2056612, 'labeled': 2056612, 'included': 2062302},
 'places': {'none': 205101, 'labeled': 205101, 'included': 487},
 'pofw': {'none': 235513, 'labeled': 235513, 'included': 235513},
 'pois': {'none': 2807137, 'labeled': 2807137, 'included': 2467828},
 'railways': {'none': 387139, 'labeled': 387139, 'included': 0},
 'roads': {'none': 37063787, 'labeled': 37063787, 'included': 28891427},
 'traffic': {'none': 5214066, 'labeled': 5214066, 'included': 1031160},
 'transport': {'none': 270918, 'labeled': 270918, 'included': 270918},
 'water': {'none': 2179499, 'labeled': 2179499, 'included': 2147549},
 'waterways': {'none': 4614306, 'labeled': 4614306, 'included': 283435}}

In [86]:
print("included/labeled")
for layer in label_count_dict:
    print(f"  {layer} : {round(label_count_dict[layer]['included']/label_count_dict[layer]['labeled'],3)}")

included/labeled
  buildings : 0.896
  landuse : 0.996
  natural : 1.003
  places : 0.002
  pofw : 1.0
  pois : 0.879
  railways : 0.0
  roads : 0.78
  traffic : 0.198
  transport : 1.0
  water : 0.985
  waterways : 0.061


### 4. New unclassified labels 

In [56]:
old_list = set(map_dict['unique_list'])

In [57]:
start = time.time()
new_list = set()

for state in state_list:
    print(state)
    state_folder_path = os.path.join(unzipped_folder_path, state)
    state_shapefile_path = os.path.join(state_folder_path, "gis_osm_*.shp")
    shapefile_finding_list = list(glob(state_shapefile_path))

    for p_shapefile in shapefile_finding_list:
        shpfile_df = gpd.read_file(p_shapefile)
        column_to_check = ["fclass", "type"]
        column_to_check = list(set(shpfile_df.columns).intersection(column_to_check))
        for colname in column_to_check:
            new_labels = set(shpfile_df[colname].unique())
            new_list = new_list.union(new_labels)
            
end = time.time()
print(f"Runtime of the program is {end - start}")

alabama
alaska
arizona
arkansas
norcal
socal
colorado
connecticut
delaware
district of columbia
florida
georgia
hawaii
idaho
illinois
indiana
iowa
kansas
kentucky
louisiana
maine
maryland
massachusetts
michigan
minnesota
mississippi
missouri
montana
nebraska
nevada
new hampshire
new jersey
new mexico
new york
north carolina
north dakota
ohio
oklahoma
oregon
pennsylvania
puerto rico
rhode island
south carolina
south dakota
tennessee
texas
united states virgin islands
utah
vermont
virginia
washington
west virginia
wisconsin
wyoming
Runtime of the program is 3437.8917973041534


In [58]:
new_list - old_list

{'mixed-use',
 'rebuilt',
 'roof_overhang',
 'cowshed',
 'stable',
 'new',
 'tomb',
 'workshop',
 'inside_store',
 'public;government',
 'mobile',
 'suite',
 'indzustrial',
 'Drive_Through',
 'boat storage house',
 'apartments (2nd floo',
 'tower 2',
 'garages;yes',
 'yurt',
 'undefined',
 'camera_surveillance',
 'fourplex',
 'resturant',
 'g',
 'fidelity',
 'Cafeteria',
 'Tractor supply',
 'private',
 'housev',
 'barn;commercial',
 'column',
 'Morse TH',
 'Pierce_County_Buildi',
 'cabinet',
 'house;detached',
 'data',
 'ski lodge',
 'fire department',
 'apartments',
 'carps',
 'public_building;yes',
 'mixed_use',
 'booth',
 'tribune',
 'Station',
 'Composting_System',
 'Carriage Lane',
 'sg',
 'Livestock_Barn',
 'apartments;garage',
 'open_air',
 'barrack',
 'clubhouse',
 'yes;university',
 'shrine',
 'stilt house',
 'retirement home',
 'narrow_gauge',
 'caboose',
 'House',
 'doghouse',
 'tes',
 'boat_house',
 'historical',
 'substation',
 'restroom',
 'Meat_Locker',
 'occupied',
 'wa

In [59]:
map_file_path = os.path.abspath(os.path.join(geo_map_path, os.pardir))

In [60]:
pd.DataFrame({"new_label":list(new_list - old_list)}).to_csv(os.path.join(map_file_path, "new_label_list.csv"))

#### 4.1 New unclassified labels in layer

In [16]:
layer_to_check = ["buildings","pois"]

In [17]:
label_dict = {"raw_data_labels": {}, "organized_data_labels": {}}

In [18]:
start = time.time()
for layer in layer_to_check:
    print(layer)
    raw_list = set()
    organized_list = set()
    
    # raw data label
    for state in state_list:
        print(state)
        state_folder_path = os.path.join(unzipped_folder_path, state)
        state_shapefile_path = os.path.join(state_folder_path, "gis_osm_"+ layer + "*.shp")
        shapefile_finding_list = list(glob(state_shapefile_path))

        for p_shapefile in shapefile_finding_list:
            shpfile_df = gpd.read_file(p_shapefile)
            column_to_check = ["fclass", "type"]
            column_to_check = list(set(shpfile_df.columns).intersection(column_to_check))

            if len(column_to_check) > 1:
                if len(shpfile_df['fclass'].unique()) > len(shpfile_df['type'].unique()):
                    column_to_check = "fclass"
                else:
                    column_to_check = "type"
            else:
                column_to_check = column_to_check[0]
                
            new_labels = set(shpfile_df[column_to_check].unique())
            raw_list = raw_list.union(new_labels)
            
    # organized data label
    for lvl1 in os.listdir(organized_data_folder_path):
        print(lvl1)
        lvl1_path = os.path.join(organized_data_folder_path, lvl1)
        for lvl2 in os.listdir(lvl1_path):
            lvl2_path = os.path.join(lvl1_path, lvl2)
            for lvl3 in os.listdir(lvl2_path):
                lvl3_path = os.path.join(lvl2_path, lvl3)
                for target_file in glob(os.path.join(lvl3_path, "*"+layer+"*.shp")):
                    shpfile_df = gpd.read_file(target_file)
                    column_to_check = ["fclass", "type"]
                    column_to_check = list(set(shpfile_df.columns).intersection(column_to_check))

                    if len(column_to_check) > 1:
                        if len(shpfile_df['fclass'].unique()) > len(shpfile_df['type'].unique()):
                            column_to_check = "fclass"
                        else:
                            column_to_check = "type"
                    else:
                        column_to_check = column_to_check[0]

                    new_labels = set(shpfile_df[column_to_check].unique())
                    organized_list = organized_list.union(new_labels)
                    
    label_dict["raw_data_labels"][layer] = list(raw_list)
    label_dict["organized_data_labels"][layer] = list(organized_list)
    
end = time.time()
print(f"Runtime of the program is {end - start}")

buildings
alabama
alaska
arizona
arkansas
norcal
socal
colorado
connecticut
delaware
district of columbia
florida
georgia
hawaii
idaho
illinois
indiana
iowa
kansas
kentucky
louisiana
maine
maryland
massachusetts
michigan
minnesota
mississippi
missouri
montana
nebraska
nevada
new hampshire
new jersey
new mexico
new york
north carolina
north dakota
ohio
oklahoma
oregon
pennsylvania
puerto rico
rhode island
south carolina
south dakota
tennessee
texas
united states virgin islands
utah
vermont
virginia
washington
west virginia
wisconsin
wyoming
busines
commercial
recreational
residential
service
pois
alabama
alaska
arizona
arkansas
norcal
socal
colorado
connecticut
delaware
district of columbia
florida
georgia
hawaii
idaho
illinois
indiana
iowa
kansas
kentucky
louisiana
maine
maryland
massachusetts
michigan
minnesota
mississippi
missouri
montana
nebraska
nevada
new hampshire
new jersey
new mexico
new york
north carolina
north dakota
ohio
oklahoma
oregon
pennsylvania
puerto rico
rhode island

In [19]:
label_dict.keys()

dict_keys(['raw_data_labels', 'organized_data_labels'])

In [20]:
label_dict['raw_data_labels'].keys()

dict_keys(['buildings', 'pois'])

In [21]:
def compare_label_list(raw_list, organized_list):
    excluded_list = []
    raw_list_clean = clean_tag_column(pd.Series(raw_list))
    for rlabel in raw_list_clean:
        for olabel in organized_list:
            if olabel in rlabel:
                break
        else:
            excluded_list.append(rlabel)
    return excluded_list

In [22]:
label_dict['excluded_data_labels'] = dict()
for layer in layer_to_check:
    raw_list = label_dict['raw_data_labels'][layer]
    raw_list = [x for x in raw_list if x is not None]
    organized_list = label_dict['organized_data_labels'][layer]
    label_dict['excluded_data_labels'][layer] = compare_label_list(raw_list, organized_list)
    

In [23]:
map_file_path = os.path.abspath(os.path.join(geo_map_path, os.pardir))

excluded_label_dict_path = os.path.join(map_file_path, "excluded_label_dict.json")

with open(excluded_label_dict_path, 'w') as fp:
    json.dump(label_dict, fp)

In [66]:
label_dict['excluded_data_labels']['buildings']

['salt_dome',
 'proposed',
 'remnant',
 'cottage_front',
 'plant',
 'printing_facility',
 'musuem',
 'commecial',
 'yew',
 'temple',
 'emergency_generator',
 'yesm',
 'ware',
 'gazebo;roof',
 'dismantled',
 'no',
 'musem',
 'none',
 'dam',
 'train_car',
 'recre',
 'vehicle',
 'spor',
 'adobe',
 'bq',
 'composting_system',
 'dugout',
 'razed',
 'grange',
 'civic',
 'ou',
 'aerobridge',
 'mausoleum',
 'roof;ye',
 'mechanical_room',
 "stable'",
 'physical_plant_(pp)',
 'yes_but_the_burgee_b',
 'dining',
 'bho',
 'refectory',
 'disused:ye',
 'true',
 'romney',
 'institutional',
 'administrative_and_r',
 'root',
 'mil',
 'sumter_psychiatry_a',
 'steam_plant',
 'cottage',
 'dt',
 'empty',
 'tipi',
 'amenity',
 'rof',
 'lucerne_ave_walk',
 'ground',
 'sidewalk',
 'transmitter',
 'john_mclain_lbra_leg',
 'gf',
 's3',
 'floating',
 'morse_th',
 'amenity_center',
 'liberty_dialysi',
 'container',
 'tower_1',
 'inu',
 'part',
 'grandstand',
 'schh',
 're',
 'outdoor_seating',
 'telescope',
 'roof

### 5. Combine files for states and layers

In [18]:
start = time.time()
start = 0
for lvl1 in os.listdir(organized_data_folder_path):
    print(lvl1)
    lvl1_path = os.path.join(organized_data_folder_path, lvl1)
    for lvl2 in os.listdir(lvl1_path):
        print("  "+lvl2)
        lvl2_path = os.path.join(lvl1_path, lvl2)
        for lvl3 in os.listdir(lvl2_path):
            print("    "+lvl3)
            if lvl3 == "house":
                start = 1
            if start == 0:
                continue
            
            lvl3_path = os.path.join(lvl2_path, lvl3)
            df_point = pd.DataFrame()
            df_polygon = pd.DataFrame()
            df_folder_path = os.path.join(combined_data_folder_path, lvl1, lvl2, lvl3)
            create_folder(df_folder_path)
            df_point_path = os.path.join(df_folder_path, lvl3+"_point.shp")
            df_line_path = os.path.join(df_folder_path, lvl3+"_line.shp")
            df_polygon_path = os.path.join(df_folder_path, lvl3+"_polygon.shp")
            
            if not os.path.exists(df_point_path):
                for target_file in glob(os.path.join(lvl3_path, "*point.shp")):
                    shpfile_df = gpd.read_file(target_file)
                    df_point = df_point.append(shpfile_df)
                if df_point.shape[0] >0:
                    df_point['geo_type'] = [type(x) for x in df_point.geometry]
                    df_line = df_point[df_point['geo_type'] == shapely.geometry.linestring.LineString]
                    if df_line.shape[0] >0:
                        df_point = df_point[df_point['geo_type'] == shapely.geometry.point.Point]
                        df_line.reset_index(inplace=True, drop=True)
                        df_point.reset_index(inplace=True, drop=True)
                        df_point = df_point.drop('geo_type', 1)
                        df_line = df_line.drop('geo_type', 1)
                        if df_point.shape[0] >0:
                            df_point.to_file(df_point_path)
                        df_line.to_file(df_line_path)
                    else:
                        df_point = df_point.drop('geo_type', 1)
                        df_point.reset_index(inplace=True, drop=True)
                        df_point.to_file(df_point_path)

            
            if not os.path.exists(df_polygon_path):
                for target_file in glob(os.path.join(lvl3_path, "*polygon.shp")):
                    shpfile_df = gpd.read_file(target_file)
                    df_polygon = df_polygon.append(shpfile_df)
                if df_polygon.shape[0] >0:
                    df_polygon.reset_index(inplace=True, drop=True)
                    df_polygon.to_file(df_polygon_path)
                
end = time.time()
print(f"Runtime of the program is {end - start}")

busines
  busines
    company
    convention_center
    factory
    industrial
    office
commercial
  food
    bakery
    beverage
    cafe
    deli
    dining_hall
    fast_food
    food
    restaurant
  leisure
    aquarium
    bar
    casino
    cinema
    museum
    theatre
    theme_park
    zoo
  lifestyle
    beauty_shop
    car_wash
    hotel
    laundry
    recycling
    self_storage
    stadium
    travel_agent
    veterinary
  other
    commercial
  shopping
    kiosk
    mall
    shop
    supermarket
recreational
  indoor
    sports_centre
    swimming_pool
  outdoor
    attraction
    bicycle_rental
    camp_site
    country_club
    dog_park
    farm
    golf_course
    graveyard
    hunting_stand
    ice_rink
    nature
    park
    picnic_site
    pitch
    playground
    shelter
    tennis_court
    tourism
    track
    viewpoint
    water
residential
  building
    building
  residential
    apartment
    dormitory
    home
    house
    residential
service
  educat

In [70]:
map_dict['map']['service']

{'education': {'college': ['college_building'],
  'childcare': ['childcare_facility', 'day_care'],
  'preschool': [],
  'kindergarten': [],
  'school': ['high_school', 'music_school', 'classroom'],
  'library': ['university_library'],
  'university': [],
  'education': ['academic', 'academic_building', 'student_center']},
 'health': {'clinic': [],
  'dentist': ['dentist_office'],
  'doctor': ['doctor'],
  'hospital': [],
  'pharmacy': [],
  'health_center': [],
  'nursing_home': [],
  'medical': [],
  'health': ['healthcare'],
  'mental_health_clinic': []},
 'religiou': {'buddhist': ['christian',
   'christian_anglican',
   'christian_catholic',
   'christian_evangelical',
   'christian_lutheran',
   'christian_methodist',
   'christian_orthodox',
   'christian_protestant',
   'hindu',
   'jewish',
   'muslim',
   'muslim_shia',
   'muslim_sunni',
   'sikh',
   'taoist',
   'cathedral',
   'chapel',
   'church',
   'monastery',
   'mosque',
   'religion',
   'presbytery',
   'religiou'

### 6. check point and polygon shpfile relationship

In [12]:
start = time.time()
for lvl1 in os.listdir(organized_data_folder_path):
    print(lvl1)
    lvl1_path = os.path.join(organized_data_folder_path, lvl1)
    for lvl2 in os.listdir(lvl1_path):
        print("  "+lvl2)
        lvl2_path = os.path.join(lvl1_path, lvl2)
        for lvl3 in os.listdir(lvl2_path):
            print("    "+lvl3)
            lvl3_path = os.path.join(lvl2_path, lvl3)
            df_point = pd.DataFrame()
            df_polygon = pd.DataFrame()
            df_folder_path = os.path.join(combined_data_folder_path, lvl1, lvl2, lvl3)

            df_point_path = os.path.join(df_folder_path, lvl3+"_point.shp")
            df_line_path = os.path.join(df_folder_path, lvl3+"_line.shp")
            df_polygon_path = os.path.join(df_folder_path, lvl3+"_polygon.shp")
            
            if not os.path.exists(df_point_path):
                for target_file in glob(os.path.join(lvl3_path, "*point.shp")):
                    shpfile_df = gpd.read_file(target_file)
                    df_point = df_point.append(shpfile_df)
                    
            if not os.path.exists(df_polygon_path):
                for target_file in glob(os.path.join(lvl3_path, "*polygon.shp")):
                    shpfile_df = gpd.read_file(target_file)
                    df_polygon = df_polygon.append(shpfile_df)
                    
            break
        break
    break
end = time.time()
print(f"Runtime of the program is {end - start}")

busines
  busines
    company
Runtime of the program is 0.0949392318725586


In [ ]:
amenity = "university"

In [38]:
point_file_path = r"D:\data\Geofabrik\organized_landmarks_combined\service\education\university\university_point.shp"
df_point = gpd.read_file(point_file_path)
polygon_file_path = r"D:\data\Geofabrik\organized_landmarks_combined\service\education\university\university_polygon.shp"
df_polygon = gpd.read_file(polygon_file_path)

In [39]:
df_point

,osm_id,code,fclass,name,tag_col_ty,geometry
0,358962262,2081,university,University of South Alabama,str,POINT (-88.17973 30.69491)
1,358974094,2081,university,Alabama Agricultural and Mechanical University,str,POINT (-86.56842 34.78390)
2,358974322,2081,university,Huntingdon College,str,POINT (-86.28525 32.35097)
3,358985169,2081,university,University of North Alabama,str,POINT (-87.67920 34.80787)
4,360195422,2081,university,Ludwig von Mises Institute,str,POINT (-85.49100 32.60672)
...,...,...,...,...,...,...
635,4291080657,2081,university,Medical College of Wisconsin-Central Wisconsin,str,POINT (-89.66383 44.96482)
636,4968819754,2081,university,Concordia University,str,POINT (-87.93287 42.56926)
637,5404250879,2081,university,Concordia University Wisconsin-Beloit Center,str,POINT (-88.98947 42.52407)
638,5676069878,2081,university,Kenosha County UW-Extension,str,POINT (-88.04467 42.56991)


In [40]:
df_polygon

,osm_id,code,fclass,name,type,tag_col_ty,geometry
0,38744956,1500,building,Palmer Hall,university,str,"POLYGON ((-86.86289 33.10384, -86.86279 33.103..."
1,38765023,1500,building,Humanities Hall,university,str,"POLYGON ((-86.86375 33.10572, -86.86370 33.105..."
2,38765026,1500,building,Morgan Hall,university,str,"POLYGON ((-86.86369 33.10604, -86.86361 33.106..."
3,38765029,1500,building,Farmer Hall,university,str,"POLYGON ((-86.86455 33.10611, -86.86455 33.106..."
4,38765036,1500,building,Myrick Hall,university,str,"POLYGON ((-86.86544 33.10602, -86.86535 33.106..."
...,...,...,...,...,...,...,...
32997,672415150,1500,building,Service Building,university,str,"POLYGON ((-105.57850 41.31498, -105.57848 41.3..."
32998,673731502,1500,building,Bureau of Mines Building,university,str,"POLYGON ((-105.58505 41.31452, -105.58504 41.3..."
32999,673731503,1500,building,Energy Innovation Center,university,str,"POLYGON ((-105.58334 41.31486, -105.58333 41.3..."
33000,480272723,2081,university,University of Wyoming,None,str,"POLYGON ((-105.58566 41.31151, -105.58565 41.3..."


In [41]:
point_set = set(df_point.osm_id.unique())

In [42]:
polygon_set = set(df_polygon.osm_id.unique())

In [43]:
point_set.intersection(polygon_set)

set()

In [44]:
df_point[df_point['name'] == 'Laramie County Community College']

,osm_id,code,fclass,name,tag_col_ty,geometry
